# Producing Project Plans
This notebook is for experimenting with having an LLM plan out a coding project.

In [ ]:
from dotenv import load_dotenv
import json
import os
import warnings
import subprocess
from openai import OpenAI
from pathlib import Path

warnings.filterwarnings("ignore")
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key:
    print("OPENAI_API_KEY is set correctly.")
else:
    print("OPENAI_API_KEY is not set.")

In [ ]:
architecture_policies = [
    {
        "name": "Modularity and Function-Oriented Design",
        "description": "Design the codebase as a collection of small, focused functions, each performing one clear task.",
        "policies": [
            "Each function should have a single responsibility.",
            "Functions should be stateless where possible.",
            "Avoid side effects (e.g., global state modifications).",
            "Write reusable and composable functions.",
        ],
    },
    {
        "name": "JSON as the Core Data Format",
        "description": "Treat JSON files as the primary medium for data input, output, and inter-process communication.",
        "policies": [
            "Input and output data should default to JSON files instead of strings.",
            "Functions should focus on reading from and writing to JSON files.",
            "Encapsulate JSON parsing and writing logic into dedicated utility functions.",
            "Ensure JSON schemas are consistent and well-documented.",
        ],
    },
    {
        "name": "Separation of Concerns",
        "description": "Keep concerns (e.g., data handling, computation, and presentation) isolated.",
        "policies": [
            "Separate data processing, business logic, and I/O operations.",
            "Avoid mixing JSON parsing with computation logic in the same function.",
            "Use interfaces or adapters for external dependencies (e.g., file systems).",
        ],
    },
    {
        "name": "Data Flow Transparency",
        "description": "Ensure clarity in how data moves through the system.",
        "policies": [
            "Functions should explicitly accept inputs and return outputs.",
            "Avoid relying on hidden state or global variables.",
            "Use clear pipelines when chaining multiple data transformations.",
        ],
    },
    {
        "name": "Minimalism (Unix Philosophy)",
        "description": "Build small, sharp tools that can be combined in pipelines.",
        "policies": [
            "Functions should do one thing well.",
            "Allow for chaining outputs to other scripts/tools.",
            "Favor simplicity over cleverness in implementation.",
            "Document command-line or API interfaces for integration.",
        ],
    },
]
policy_list = list(
    set(policy for item in architecture_policies for policy in item.get("policies", []))
)
arch_policies = "\n\t- ".join(policy_list)

In [ ]:
abstraction_policies = [
    {
        "name": "Only Abstract Where There Is Real Complexity",
        "description": (
            "An 'abstraction' that merely renames or shallowly encapsulates something "
            "without reducing real complexity is worse than no abstraction at all."
        ),
        "policies": [
            "Resist the urge to introduce extra layers unless they simplify or hide significant complexity.",
            "Ensure each abstraction brings tangible value (e.g., consolidating behavior or unifying concepts).",
        ],
    },
    {
        "name": "Align with the Domain's Terminology and Concepts",
        "description": (
            "Abstractions should mirror real-world or domain-specific entities to avoid "
            "cognitive dissonance and confusion."
        ),
        "policies": [
            "Use domain language for classes, methods, and modules.",
            "If naming feels forced, question whether the abstraction is really needed.",
        ],
    },
    {
        "name": "Avoid 'Mere Wrappers'",
        "description": (
            "Simply forwarding calls or adding a superficial layer on top of an existing "
            "API doesn't constitute a meaningful abstraction."
        ),
        "policies": [
            "Ensure your wrapper adds domain logic or enforces rules.",
            "Eliminate redundant layers that add no value beyond delegation.",
        ],
    },
    {
        "name": "Make It Easy to Replace or Update Implementations",
        "description": (
            "A well-designed abstraction should be flexible enough to allow for easy "
            "swapping of underlying implementations."
        ),
        "policies": [
            "Favor loose coupling and clear contracts over shared mutable state.",
            "Ensure external code doesn't break if internals change or evolve.",
        ],
    },
    {
        "name": "Model Behavior, Not Just Data",
        "description": (
            "Abstractions are most useful when they encapsulate behaviors or state "
            "transitions, not just fields."
        ),
        "policies": [
            "Group domain logic (validations, transformations) with the data it affects.",
            "Use simpler structures (e.g., dicts, tuples) for data-only needs.",
        ],
    },
    {
        "name": "Keep Abstractions at Appropriate Levels of Granularity",
        "description": (
            "Too many thin layers or one monolithic abstraction can both be detrimental. "
            "Strike a balance that aligns with the domain."
        ),
        "policies": [
            "Avoid stacking multiple tiny wrappers that only pass data through.",
            "Break large abstractions into manageable pieces based on cohesive functionality.",
        ],
    },
    {
        "name": "Be Wary of Cargo-Culting Patterns",
        "description": (
            "Blindly using popular patterns or structures without addressing specific "
            "problems can lead to unnecessary complexity."
        ),
        "policies": [
            "Adopt patterns only after identifying a clear need they fulfill.",
            "Question defaults—avoid implementing a pattern 'just because' it's popular.",
        ],
    },
]
policy_list = list(
    set(policy for item in abstraction_policies for policy in item.get("policies", []))
)
abstraction_policies = "\n\t- ".join(policy_list)

In [ ]:
plan_format_policies = [
    {
        "name": "MD040",
        "description": "Fenced code blocks should have a language specified",
        "policies": [
            "Fenced code blocks should have a language identifier.",
            "Use the appropriate language identifier for the code block.",
            "Ensure the language identifier is supported by the markdown parser.",
        ],
    },
    {
        "name": "Avoid non-plan content",
        "description": "Ensure that the response is restricted to the plan, without additional commentary.",
        "policies": [
            "Avoid adding non-plan content to the response.",
        ],
    },
]
policy_list = list(
    set(policy for item in plan_format_policies for policy in item.get("policies", []))
)
format_policies = "\n\t- ".join(policy_list)

In [ ]:
project_plan_template = """
#       PROJECT PLAN  TEMPLATE  #

## 1. Architectural Style
<choose_one>
- Layered Architecture: Presentation, Business Logic, Data Access, Database
- Microservices Architecture: Independent services for specific capabilities
- Modular Monolith: Independent modules, shared deployment
- Event-Driven Architecture: Event producers, consumers, queues

## 2. Component Interface Definition
- Accepted Data: [Data input]
- Returned Data: [Data Output]

## 3. Domain-Driven Design (DDD)
Define bounded contexts:
- User Context: [Responsibilities]
- Order Context: [Responsibilities]
- Payment Context: [Responsibilities]

## 4. Dependency Management
- Dependencies Between Components: [Dependencies]
- Cyclic Dependency Avoidance: [Strategies]
- Dependency Injection: [Yes/No, describe]

## 5. Data Flow Between Sub-Components
Describe data flow:
- Sequence Diagram: [Interactions as ASCII art]
- Data Flow Diagram: [Flow as ASCII art]
- Component Diagram: [Dependencies between components as ASCII art]

## 6. Component Documentation
Document each component:
- **Name:** [Unique Identifier]
- **Purpose:** [Description of purpose]
- **Inputs:** [Expected inputs]
- **Outputs:** [Expected outputs]
- **Dependencies:** [List dependencies]

#       END OF TEMPLATE      #
"""

In [ ]:
project_idea_paths = Path("project_ideas").glob("*.json")
project_ideas = {"_".join(fn.stem.split("_")[1:]): json.load(open(fn)) for fn in project_idea_paths}

In [ ]:
project_plan_prompt_preamble = """ 
You are a software engineer tasked with designing a new project.
Your goal is to create a project plan that follows best practices in software architecture and coding.
"""

In [ ]:
def format_plan(project_name, plan):
    "Removes template header/footer and adds title."
    start = plan.find("## 1.")
    end = plan.rfind("END OF TEMPLATE")
    plan = plan[start:end].strip().strip("#").strip()
    return f"# {project_name}\n## {plan}\n"

In [ ]:
client = OpenAI(api_key=openai_api_key)

for project_name, project_idea in project_ideas.items():
    project_plan_prompt = f"""{project_plan_prompt_preamble}
    Your project should adhere to the following policies::\n {arch_policies}\n
    Solve this problem:\n {project_idea}\n
    Follow this template (removing the template header/footer):\n {project_plan_template}\n
    Format the plan according to these policies:\n {format_policies}\n
    """
    response = client.chat.completions.create(
        model="o4-mini", reasoning={"effort": "medium"},
        messages=[{"role": "user", "content": project_plan_prompt}]
    )
    plan = format_plan(project_name, response.choices[0].message.content)

    # Create directory and write file
    projects_path = Path("projects")
    projects_path.mkdir(exist_ok=True)
    project_path = projects_path / project_name
    project_path.mkdir(exist_ok=True)
    (project_path / "plan.md").write_text(plan)

    # format the plan to be good markdown
    subprocess.run(["mdformat", str(project_path / "plan.md")])